## Define fields

In [ ]:
from josie.fluid.fields import FluidFields
from josie.fields import Fields

class SigmaFields(FluidFields):
    # Conservative
    rho = 0
    rhoU = 1
    rhoV = 2
    rhoY = 3
    rhoYw = 4
    rhoa = 5
    rhoz = 6
    
    # Auxiliary
    U = 7
    V = 8
    P = 9
    c = 10
    Y = 11
    w = 12
    alpha = 13
    z = 14

    
class SigmaConsFields(Fields):
    rho = 0
    rhoU = 1
    rhoV = 2
    rhoY = 3
    rhoYw = 4
    rhoa = 5
    rhoZ = 6

## Define State

In [ ]:
from josie.fluid.state import SingleFluidState
from josie.state import SubsetState

class SigmaConsState(SubsetState):
    full_state_fields = SigmaFields
    fields = SigmaConsFields
    
class SigmaState(SingleFluidState):
    fields = SigmaFields
    cons_state = SigmaConsState
    


## Define Barotropic EOS

In [ ]:
class Barotropic:
    """
    .. math::
    
        \pressure = a\rho^b
    """
    
    def __init__(self, a, b, surface_tension):
        self._constant = a
        self._exp = b
        self._sigma = surface_tension
    
    def p(self, rho):
        return self._constant * rho ** self._exp
    
    def rho(self, p):
        return (p / self._constant) ** (1/self._exp)
    
    def sound_velocity(self, rho, p):
        return self._constant * self._exp * rho ** (self._exp - 1)

## Define Mixture EOS

In [ ]:
from josie.twofluid.state import PhasePair

class MixtureEOS(PhasePair):
    def __init__(self, eos1, eos2, surface_tension):
        super().__init__(eos1, eos2)
        self.surface_tension = surface_tension

## Define Problem

In [ ]:
from josie.problem import Problem
from josie.dimension import MAX_DIMENSIONALITY
from josie.math import Direction

class SigmaProblem(Problem):
    def __init__(self, eos: MixtureEOS):
        self.eos = eos
        
    def F(self, cells):
        values = cells.values.view(SigmaState)
        nx, ny, _ = values.shape
        
        # Flux tensor
        F = np.zeros(
            (
                nx,
                ny,
                len(SigmaConsFields),
                MAX_DIMENSIONALITY,
            )
        )
        
        fields = values.fields
                  
        rho = values[..., fields.alpha]
        rhoU = values[..., fields.rhoU]
        rhoV = values[..., fields.rhoV]
        rhoY = values[..., fields.rhoY]
        rhoYw = values[..., fields.rho]
        rhoa = values[..., fields.rhoa]
        
        U = values[..., fields.U]
        V = values[..., fields.V]
        P = values[..., fields.P]
        
       
        F[..., fields.rho, Direction.X] = rhoU
        F[..., fields.rho, Direction.Y] = rhoV
        F[..., fields.rhoU, Direction.X] = rhoU * U + P
        F[..., fields.rhoU, Direction.Y] = rhoU * V
        F[..., fields.rhoV, Direction.X] = rhoV * U
        F[..., fields.rhoV, Direction.Y] = rhoV * V + P
        F[..., fields.rhoY, Direction.X] = rhoY * U
        F[..., fields.rhoY, Direction.Y] = rhoY * V
        F[..., fields.rhoYw, Direction.X] = rhoYw * U
        F[..., fields.rhoYw, Direction.Y] = rhoYw * V
        F[..., fields.rhoa, Direction.X] = rhoa * U
        F[..., fields.rhoa, Direction.Y] = rhoa * V
        F[..., fields.rhoz, Direction.X] = rhoz * U
        F[..., fields.rhoz, Direction.Y] = rhoz * V
        
        return F
        
        

## Define Scheme

In [ ]:
import numpy as np

from josie.euler.schemes import EulerScheme

class SigmaScheme(EulerScheme):
    def __init__(self, eos: TwoPhaseEOS):
        super().__init_(SigmaProblem(eos))
        
    def post_step(self, cells):
        values = cells.values.view(SigmaState)
        fields = values.fields
        
        rho = values[..., fields.alpha]
        rhoU = values[..., fields.rhoU]
        rhoV = values[..., fields.rhoV]
        rhoY = values[..., fields.rhoY]
        rhoYw = values[..., fields.rhoYw]
        rhoa = values[..., fields.rhoa]
        rhoz = values[..., fields.rhoz]
        
        U = rhoU / rho
        V = rhoV / rho
        Y = rhoY / rho
        Y2 = 1 - Y
        w = rhoYw / rho / Y
        alpha = rhoa / rho
        alpha2 = 1 - alpha
        z = rhoz / rho
        Sigma = (rho ** (1/3)) * (z ** (2/3))
        
        surface_tension = self.problem.eos.surface_tension
        
        eos1 = self.problem.eos[0]
        rho1 = rho * Y / alpha
        p1 = eos1.p(rho1)
        c1 = eos1.sound_velocity(rho1, p1)
        
        
        rho2 = rho * Y2 / alpha2 
        eos2 = self.problem.eos[1]
        p2 = eos2.p(rho2)
        c2 = eos2.sound_velocity(rho1, p1)
        
        P = alpha * p1 + alpha2 * p2 + 0.5 * rhoYw**2 \
            - 2/3 * surface_tension * Sigma
        
        c = np.sqrt(Y * c1**2 + Y2 * c2**2  + rho * (Y * w)**2 \
            + 2/9 * surface_tension * Sigma / rho)
               
        values[..., fields.U] = U
        values[..., fields.V] = V
        values[..., fields.P] = P
        values[..., fields.c] = c
        values[..., fields.Y] = Y
        values[..., fields.w] = w
        values[..., fields.alpha] = alpha
        values[..., fields.z] = z

## Rusanov Scheme

In [ ]:
from josie.euler.schemes import Rusanov as EulerRusanov

class Rusanov(EulerRusanov):
    def F(self, cells, neighs):

        Q_L = cells.values.view(SigmaState)
        Q_R = neighs.values.view(SigmaState)

        fields = SigmaState.fields

        FS = np.zeros_like(Q_L).view(SigmaState)

        # Get normal velocities
        U_L = self.compute_U_norm(Q_L, neighs.normals)
        U_R = self.compute_U_norm(Q_R, neighs.normals)

        # Speed of sound
        c_L = Q_L[..., fields.c]
        c_R = Q_R[..., fields.c]

        sigma = self.compute_sigma(U_L, U_R, c_L, c_R)

        DeltaF = 0.5 * (self.problem.F(cells) + self.problem.F(neighs))

        # This is the flux tensor dot the normal
        DeltaF = np.einsum("...kl,...l->...k", DeltaF, neighs.normals)

        # First four variables of the total state are the conservative
        # variables (rho, rhoU, rhoV, rhoE)
        Q_L_cons = Q_L.get_conservative()
        Q_R_cons = Q_R.get_conservative()

        DeltaQ = 0.5 * sigma * (Q_R_cons - Q_L_cons)

        FS.set_conservative(
            neighs.surfaces[..., np.newaxis] * (DeltaF - DeltaQ)
        )

        return FS

## Define Solver

In [ ]:
from josie.solver import Solver

class SigmaSolver(Solver):
    def __init__(self, mesh, scheme):

        super().__init__(mesh, SigmaState, scheme)
